In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer

In [2]:
# Load the data and build the model

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)
X_new = X_test[:3]

In [3]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = Sequential()
    model.add(InputLayer(input_shape=input_shape))
    for i in range(n_hidden):
        model.add(Dense(units=n_neurons, activation='relu'))
    model.add(Dense(1))
    
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [4]:
keras_model = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [5]:
keras_model.fit(
    X_train, 
    y_train, 
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 11610 samples, validate on 3870 samples
Epoch 1/50
11610/11610 [==============================] - 3s 222us/sample - loss: 1.4056 - val_loss: 0.6325
Epoch 2/50
11610/11610 [==============================] - 1s 70us/sample - loss: 0.6933 - val_loss: 0.5798
Epoch 3/50
11610/11610 [==============================] - 1s 87us/sample - loss: 0.5502 - val_loss: 0.5079
Epoch 4/50
11610/11610 [==============================] - 1s 71us/sample - loss: 0.4827 - val_loss: 0.4812
Epoch 5/50
11610/11610 [==============================] - 1s 72us/sample - loss: 0.4608 - val_loss: 0.4651
Epoch 6/50
11610/11610 [==============================] - 1s 70us/sample - loss: 0.4460 - val_loss: 0.4536
Epoch 7/50
11610/11610 [==============================] - 1s 72us/sample - loss: 0.4351 - val_loss: 0.4457
Epoch 8/50
11610/11610 [==============================] - 1s 76us/sample - loss: 0.4269 - val_loss: 0.4365
Epoch 9/50
11610/11610 [==============================] - 1s 89us/sample - loss: 0.4199 - val_

In [18]:
mse_test = keras_model.score(X_test, y_test, verbose=2)
mse_test

5160/1 - 0s - loss: 0.4073


-0.3860518298407858

In [22]:
y_pred = keras_model.predict(X_new)
y_pred

array([3.0468493, 3.0544968, 4.635125 ], dtype=float32)

In [25]:
# Use Randomized search to find better hyperparameters
import numpy as np
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [26]:
params = {
    'n_hidden':[0,1,2,3],
    'n_neurons':np.arange(1,100),
    'learning_rate':reciprocal(3e-4, 3e-2)
}

In [36]:
random_search = RandomizedSearchCV(keras_model, params, n_iter=10, cv=3)

In [37]:
random_search.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data = (X_valid, y_valid),
    callbacks = [keras.callbacks.EarlyStopping(patience=10)]
)

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 2s 212us/sample - loss: 0.7740 - val_loss: 0.5326
Epoch 2/100
7740/7740 [==============================] - 1s 99us/sample - loss: 0.4715 - val_loss: 0.4660
Epoch 3/100
7740/7740 [==============================] - 1s 100us/sample - loss: 0.4494 - val_loss: 0.4636
Epoch 4/100
7740/7740 [==============================] - 1s 100us/sample - loss: 0.4270 - val_loss: 0.4337
Epoch 5/100
7740/7740 [==============================] - 1s 99us/sample - loss: 0.4227 - val_loss: 0.4104
Epoch 6/100
7740/7740 [==============================] - 1s 98us/sample - loss: 0.3967 - val_loss: 0.4007
Epoch 7/100
7740/7740 [==============================] - 1s 105us/sample - loss: 0.3872 - val_loss: 0.3931
Epoch 8/100
7740/7740 [==============================] - 1s 98us/sample - loss: 0.3777 - val_loss: 0.3841
Epoch 9/100
7740/7740 [==============================] - 1s 102us/sample - loss: 0.3758 - val_loss: 

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 2s 204us/sample - loss: 0.8963 - val_loss: 1.2351
Epoch 2/100
7740/7740 [==============================] - 1s 119us/sample - loss: nan - val_loss: nan
Epoch 3/100
7740/7740 [==============================] - 1s 131us/sample - loss: nan - val_loss: nan
Epoch 4/100
7740/7740 [==============================] - 1s 103us/sample - loss: nan - val_loss: nan
Epoch 5/100
7740/7740 [==============================] - 1s 99us/sample - loss: nan - val_loss: nan
Epoch 6/100
7740/7740 [==============================] - 1s 95us/sample - loss: nan - val_loss: nan
Epoch 7/100
7740/7740 [==============================] - 1s 98us/sample - loss: nan - val_loss: nan
Epoch 8/100
7740/7740 [==============================] - 1s 131us/sample - loss: nan - val_loss: nan
Epoch 9/100
7740/7740 [==============================] - 1s 158us/sample - loss: nan - val_loss: nan
Epoch 10/100
7740/7740 [================

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 191us/sample - loss: 0.7210 - val_loss: 0.5208
Epoch 2/100
7740/7740 [==============================] - 1s 99us/sample - loss: 0.6056 - val_loss: 0.4407
Epoch 3/100
7740/7740 [==============================] - 1s 100us/sample - loss: 0.5280 - val_loss: 0.4350
Epoch 4/100
7740/7740 [==============================] - 1s 153us/sample - loss: 0.5009 - val_loss: 0.4464
Epoch 5/100
7740/7740 [==============================] - 1s 162us/sample - loss: 0.5217 - val_loss: 0.4226
Epoch 6/100
7740/7740 [==============================] - 1s 122us/sample - loss: 0.4317 - val_loss: 0.5082
Epoch 7/100
7740/7740 [==============================] - 1s 106us/sample - loss: 0.5147 - val_loss: 0.4232
Epoch 8/100
7740/7740 [==============================] - 1s 102us/sample - loss: 0.4107 - val_loss: 0.3955
Epoch 9/100
7740/7740 [==============================] - 1s 106us/sample - loss: 0.3986 - val_los

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 2s 237us/sample - loss: 1.3724 - val_loss: 0.7126
Epoch 2/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.6425 - val_loss: 0.6464
Epoch 3/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.5883 - val_loss: 0.5877
Epoch 4/100
7740/7740 [==============================] - 1s 88us/sample - loss: 0.5528 - val_loss: 0.5600
Epoch 5/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.5280 - val_loss: 0.5361
Epoch 6/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.5097 - val_loss: 0.5211
Epoch 7/100
7740/7740 [==============================] - 1s 90us/sample - loss: 0.4953 - val_loss: 0.5088
Epoch 8/100
7740/7740 [==============================] - 1s 88us/sample - loss: 0.4847 - val_loss: 0.4981
Epoch 9/100
7740/7740 [==============================] - 1s 88us/sample - loss: 0.4754 - val_loss: 0.49

7740/7740 [==============================] - 1s 92us/sample - loss: 0.3373 - val_loss: 0.3519
Epoch 78/100
7740/7740 [==============================] - 1s 95us/sample - loss: 0.3370 - val_loss: 0.3512
Epoch 79/100
7740/7740 [==============================] - 1s 90us/sample - loss: 0.3368 - val_loss: 0.3454
Epoch 80/100
7740/7740 [==============================] - 1s 89us/sample - loss: 0.3350 - val_loss: 0.3425
Epoch 81/100
7740/7740 [==============================] - 1s 96us/sample - loss: 0.3338 - val_loss: 0.3418
Epoch 82/100
7740/7740 [==============================] - 1s 88us/sample - loss: 0.3339 - val_loss: 0.3424
Epoch 83/100
7740/7740 [==============================] - 1s 96us/sample - loss: 0.3331 - val_loss: 0.3408
Epoch 84/100
7740/7740 [==============================] - 1s 93us/sample - loss: 0.3318 - val_loss: 0.3410
Epoch 85/100
7740/7740 [==============================] - 1s 89us/sample - loss: 0.3304 - val_loss: 0.3403
Epoch 86/100
7740/7740 [==========================

...

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 166us/sample - loss: 0.8705 - val_loss: 0.5911
Epoch 2/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.5771 - val_loss: 0.5040
Epoch 3/100
7740/7740 [==============================] - 1s 85us/sample - loss: 0.4770 - val_loss: 0.4608
Epoch 4/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.4337 - val_loss: 0.4581
Epoch 5/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.4158 - val_loss: 0.4290
Epoch 6/100
7740/7740 [==============================] - 1s 88us/sample - loss: 0.4048 - val_loss: 0.4172
Epoch 7/100
7740/7740 [==============================] - 1s 94us/sample - loss: 0.3934 - val_loss: 0.4121
Epoch 8/100
7740/7740 [==============================] - 1s 88us/sample - loss: 0.3864 - val_loss: 0.4166
Epoch 9/100
7740/7740 [==============================] - 1s 90us/sample - loss: 0.3808 - val_loss: 0.40

Epoch 77/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.2730 - val_loss: 0.2989
Epoch 78/100
7740/7740 [==============================] - 1s 88us/sample - loss: 0.2722 - val_loss: 0.2961
Epoch 79/100
7740/7740 [==============================] - 1s 84us/sample - loss: 0.2696 - val_loss: 0.2949
Epoch 80/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.2696 - val_loss: 0.3072
Epoch 81/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.2717 - val_loss: 0.2928
Epoch 82/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.2701 - val_loss: 0.2943
Epoch 83/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.2693 - val_loss: 0.2911
Epoch 84/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.2673 - val_loss: 0.2933
Epoch 85/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.2677 - val_loss: 0.3019
Epoch 86/100
7740/7740 [=============

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 171us/sample - loss: 1.1229 - val_loss: 0.7567
Epoch 2/100
7740/7740 [==============================] - 1s 87us/sample - loss: 1.2985 - val_loss: 0.6370
Epoch 3/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.7362 - val_loss: 0.5802
Epoch 4/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.6175 - val_loss: 0.5831
Epoch 5/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.7099 - val_loss: 0.4570
Epoch 6/100
7740/7740 [==============================] - 1s 89us/sample - loss: 0.4582 - val_loss: 0.4339
Epoch 7/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.4378 - val_loss: 0.4287
Epoch 8/100
7740/7740 [==============================] - 1s 88us/sample - loss: 0.4279 - val_loss: 0.4149
Epoch 9/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.4204 - val_loss: 0.41

Epoch 77/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.2989 - val_loss: 0.3026
Epoch 78/100
7740/7740 [==============================] - 1s 88us/sample - loss: 0.2947 - val_loss: 0.2959
Epoch 79/100
7740/7740 [==============================] - 1s 85us/sample - loss: 0.2924 - val_loss: 0.2990
Epoch 80/100
7740/7740 [==============================] - 1s 88us/sample - loss: 0.2886 - val_loss: 0.3001
Epoch 81/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.2887 - val_loss: 0.3022
Epoch 82/100
7740/7740 [==============================] - 1s 85us/sample - loss: 0.2882 - val_loss: 0.2977
Epoch 83/100
7740/7740 [==============================] - 1s 87us/sample - loss: 0.2886 - val_loss: 0.2935
Epoch 84/100
7740/7740 [==============================] - 1s 86us/sample - loss: 0.2869 - val_loss: 0.2984
Epoch 85/100
7740/7740 [==============================] - 1s 88us/sample - loss: 0.2853 - val_loss: 0.3037TA: 0s - loss: 0.285
Epoch 86/100
7740

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x1a46ad25d0>, as the constructor either does not set or modifies parameter learning_rate

In [38]:
random_search.best_params_

{'learning_rate': 0.007093089504178925, 'n_hidden': 2, 'n_neurons': 56}

In [40]:
random_search.best_score_

-0.3121812332764871

In [41]:
random_search

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'